In [5]:
#import Libraries
import sympy as sp
import numpy as np

def lagrange_opt_mixed(of, equality_constraints, inequality_constraints, maximize=False):
    """
    Implements Lagrange multiplier for optimization problem with mixed constraints.

    Parameters:
        of (string): Objective function.
        equality_constraints (list): Equality constraints.
        inequality_constraints (list): Inequality constraints in the form `h(x, y) <= 0`.
        maximize (boolean): Default value is False.

    Returns:
        optimal (tuple): Optimal solution (x, y).
    """
    # Define the objective function
    f = sp.sympify(of)
    
    # Define the list of equality constraints
    equality_constraints_list = [sp.sympify(eq) for eq in equality_constraints]

    # Define the list of inequality constraints
    inequality_constraints_list = [sp.sympify(con) for con in inequality_constraints]

    # Define Lagrange multipliers for equality constraints
    lagrange_multipliers_eq = [sp.symbols(f'lambda_eq{i}') for i in range(len(equality_constraints_list))]

    # Define Lagrange multipliers for inequality constraints
    lagrange_multipliers_ineq = [sp.symbols(f'lambda_ineq{i}') for i in range(len(inequality_constraints_list))]

    # Define the Lagrangian function
    lagrangian_eq = f + sum([lam * eq for lam, eq in zip(lagrange_multipliers_eq, equality_constraints_list)])
    lagrangian_ineq = lagrangian_eq + sum([lam * con for lam, con in zip(lagrange_multipliers_ineq, inequality_constraints_list)])
    
    variables = [sp.symbols('x'), sp.symbols('y')]
    lagrange_vars_eq = lagrange_multipliers_eq
    lagrange_vars_ineq = lagrange_multipliers_ineq

    # Compute gradients w.r.t. the variables
    gradients = [sp.diff(lagrangian_ineq, var) for var in variables]
    
    eqs = gradients + [lam_eq * eq for lam_eq, eq in zip(lagrange_multipliers_eq, equality_constraints_list)] + \
          [lam_ineq * con for lam_ineq, con in zip(lagrange_multipliers_ineq, inequality_constraints_list)]
    
    varss = variables + lagrange_vars_eq + lagrange_vars_ineq

    # Solve the system of equations
    solution = sp.solve(eqs, varss)
    
    # Extract x, y values from solutions
    points = [(point[0], point[1]) for point in solution]
    
    # Remove the points that don't satisfy the lambda conditions for inequalities
    final_points = []
    if maximize:
        for sol in points:
            lambdas = sol[2+len(equality_constraints_list):]
            if all(con <= 0 and lam >= 0 for con, lam in zip(inequality_constraints_list, lambdas)):
                final_points.append((sol[0], sol[1]))
    else:
        for sol in points:
            lambdas = sol[2+len(equality_constraints_list):]
            if all(con <= 0 and lam <= 0 for con, lam in zip(inequality_constraints_list, lambdas)):
                final_points.append((sol[0], sol[1]))

    # Remove the points that don't satisfy the equality constraints
    final_points_eq = []
    for point in final_points:
        satisfies_equality_constraints = all(eq.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) == 0
                                             for eq in equality_constraints_list)
        if satisfies_equality_constraints:
            final_points_eq.append(point)

    # Get the point which minimizes/maximizes the function
    if maximize:
        optimal = (
        final_points_eq[np.argmax([f.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) for point in final_points_eq])])
    else:
        optimal = (
        final_points_eq[np.argmin([f.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) for point in final_points_eq])])

    return optimal




In [6]:
#Example 1 in the sheet
objective_function = 'x**2 - y**2'
constraints_equality = ['x + 2*y + 1']
constraints_inequality = ['x - y - 3']

solution = lagrange_opt_mixed(objective_function, constraints_equality, constraints_inequality)
print("Optimal solution:", solution)

Optimal solution: (1/3, -2/3)


In [7]:
#Example 2 in the sheet
objective_function = '(x-1)**2 + y-2'
constraints_equality = ['y - x - 1']
constraints_inequality = ['x + y - 2']

solution = lagrange_opt_mixed(objective_function, constraints_equality, constraints_inequality)
print("Optimal solution:", solution)

Optimal solution: (1/2, 3/2)
